In [9]:
import os
import sys
import numpy as np
import time
import warnings

# Define paths
notebooks_path = os.path.abspath(os.getcwd()) 
src_path = os.path.abspath(os.path.join(notebooks_path, "../src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import modules
from Multi_dimension.Multidimension_trees import *
from Multi_dimension.Multidimension_solver import *
from Multi_dimension.Multidimension_adapted_empirical_measure import *

from Measure_sampling.Gen_Path_and_AdaptedTrees import generate_adapted_tree
from trees.Tree_Node import *
from trees.TreeAnalysis import *
from trees.TreeVisualization import *
from trees.Save_Load_trees import *
from trees.Tree_AWD_utilities import *
from trees.Build_trees_from_paths import build_tree_from_paths

from adapted_empirical_measure.AEM_grid import *
from adapted_empirical_measure.AEM_kMeans import *
from benchmark_value_gaussian.Comp_AWD2_Gaussian import *
from awd_trees.Gurobi_AOT import *
from awd_trees.Nested_Dist_Algo import compute_nested_distance

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

### Generate Paths for d=1

In [2]:
# Normalization flag
normalize = False

# Define factor matrices
L0 = np.array([[1, 0, 0, 0], [1, 2, 0, 0], [1, 2, 3, 0], [1,2,3, 4]])
A0 = L0 @ L0.T
L = L0 / np.sqrt(np.trace(A0)) if normalize else L0
A = L @ L.T

M0 = np.array([[1, 0, 0, 0], [2, 1, 0, 0], [3, 2, 1, 0], [4, 3, 2, 1]])
B0 = M0 @ M0.T
M = M0 / np.sqrt(np.trace(B0)) if normalize else M0
B = M @ M.T

# Parameters
d = 1
T = 4
dim = d * T
n_sample_plot = 400

# Generate all noise samples at once
noise1 = np.random.normal(size=(n_sample_plot, dim))
noise2 = np.random.normal(size=(n_sample_plot, dim))

# Apply transformations
X_increments = (noise1 @ L.T).reshape(n_sample_plot, T, d)
Y_increments = (noise2 @ M.T).reshape(n_sample_plot, T, d)

# Prepend zeros along the time axis
X_paths = np.concatenate([np.zeros((n_sample_plot, 1, d)), X_increments], axis=1)
Y_paths = np.concatenate([np.zeros((n_sample_plot, 1, d)), Y_increments], axis=1)

### Compute Nested Distance (Multi-Dimensional Framework for $\mathbb{R}^{1\cdot T}$)

In [3]:
# Adapt empirical measures
adapted_X, adapted_weights_X = multidim_uniform_empirical_grid_measure(X_paths, use_weights=True)
adapted_Y, adapted_weights_Y = multidim_uniform_empirical_grid_measure(Y_paths, use_weights=True)

# Build trees
adapted_tree_1 = multidim_build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = multidim_build_tree_from_paths(adapted_Y, adapted_weights_Y)

# Compute nested distance
max_depth = multidim_get_depth(adapted_tree_1)
start_time = time.time()
distance_pot = multidim_compute_nested_distance(adapted_tree_1, adapted_tree_2, max_depth, power=2)
end_time = time.time()

print("Nested distance multi dim:", distance_pot)
print("Computation time: {:.4f} seconds".format(end_time - start_time))

Depth 0: 100%|██████████| 1/1 [00:00<00:00, 1207.34pair/s]

Nested distance multi dim: 34.928912989478626
Computation time: 39.3941 seconds


### Compute Nested Distance (Original Code for $\mathbb{R}^{T}$)

In [4]:
# Adapt empirical measures
X, Y = np.squeeze(X_paths, axis=-1), np.squeeze(Y_paths, axis=-1)
adapted_X, adapted_weights_X = uniform_empirical_grid_measure(X, use_weights=True)
adapted_Y, adapted_weights_Y = uniform_empirical_grid_measure(Y, use_weights=True)

# Build trees
adapted_tree_1 = build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = build_tree_from_paths(adapted_Y, adapted_weights_Y)

# Compute nested distance
max_depth = get_depth(adapted_tree_1)
start_time = time.time()
distance_pot = compute_nested_distance(adapted_tree_1, adapted_tree_2, max_depth, method="solver_lp_pot", return_matrix=False, lambda_reg=0, power=2)
elapsed_time_pot = time.time() - start_time

print("Nested distance single dim:", distance_pot)
print("Computation time: {:.4f} seconds".format(elapsed_time_pot))

Depth 0: 100%|██████████| 1/1 [00:00<00:00, 1648.06it/s]

Nested distance single dim: 34.92891298947862
Computation time: 31.9968 seconds


### Theoretical Nested Distance

In [5]:
a, b = np.zeros(dim), np.zeros(dim)
distance_aw2 = adapted_wasserstein_squared(a, A, b, B, d, T)

print("Adapted Wasserstein Squared Distance for custom Gaussian process:", distance_aw2)

Adapted Wasserstein Squared Distance for custom Gaussian process: 30.0


## For d = 2

In [14]:
# Parameters
d = 2
T = 2
dim = d * T
n_sample_plot = 1200

# Generate all noise samples at once
noise1 = np.random.normal(size=(n_sample_plot, dim))
noise2 = np.random.normal(size=(n_sample_plot, dim))

# Apply transformations
X_increments = (noise1 @ L.T).reshape(n_sample_plot, T, d)
Y_increments = (noise2 @ M.T).reshape(n_sample_plot, T, d)

# Prepend zeros along the time axis
X_paths = np.concatenate([np.zeros((n_sample_plot, 1, d)), X_increments], axis=1)
Y_paths = np.concatenate([np.zeros((n_sample_plot, 1, d)), Y_increments], axis=1)

In [15]:
# Adapt empirical measures
adapted_X, adapted_weights_X = multidim_uniform_empirical_grid_measure(X_paths, use_weights=True)
adapted_Y, adapted_weights_Y = multidim_uniform_empirical_grid_measure(Y_paths, use_weights=True)

# Build trees
adapted_tree_1 = multidim_build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = multidim_build_tree_from_paths(adapted_Y, adapted_weights_Y)

# Compute nested distance
max_depth = multidim_get_depth(adapted_tree_1)
start_time = time.time()
distance_pot = multidim_compute_nested_distance(adapted_tree_1, adapted_tree_2, max_depth, power=2)
end_time = time.time()

print("Nested distance multi dim:", distance_pot)
print("Computation time: {:.4f} seconds".format(end_time - start_time))

Depth 0: 100%|██████████| 1/1 [00:00<00:00,  6.72pair/s]

Nested distance multi dim: 9.90500337949822
Computation time: 958.1588 seconds


In [16]:
# Adapt empirical measures
adapted_X, adapted_weights_X = multidim_empirical_k_means_measure_new(X_paths, use_weights=True)
adapted_Y, adapted_weights_Y = multidim_empirical_k_means_measure_new(Y_paths, use_weights=True)

# Build trees
adapted_tree_1 = multidim_build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = multidim_build_tree_from_paths(adapted_Y, adapted_weights_Y)

# Compute nested distance
max_depth = multidim_get_depth(adapted_tree_1)
start_time = time.time()
distance_pot = multidim_compute_nested_distance(adapted_tree_1, adapted_tree_2, max_depth, power=2)
end_time = time.time()

print("Nested distance multi dim:", distance_pot)
print("Computation time: {:.4f} seconds".format(end_time - start_time))

Depth 0: 100%|██████████| 1/1 [00:00<00:00, 15.05pair/s]

Nested distance multi dim: 13.934120626835117
Computation time: 335.4728 seconds


In [17]:
a, b = np.zeros(dim), np.zeros(dim)
distance_aw2 = adapted_wasserstein_squared(a, A, b, B, d, T)

print("Adapted Wasserstein Squared Distance for custom Gaussian process:", distance_aw2)

Adapted Wasserstein Squared Distance for custom Gaussian process: 22.02336710106509
